In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

## EM-алгоритм

In [ ]:
np.random.seed(43)

n = 1000
tau = 0.8

n1 = int(tau * n)
mu1 = np.array([-1.0, -0.5])
sigma1 = np.array([
    [2.0, 0.0],
    [0.0, 2.0],
]
)

n2 = int((1-tau) * n)
mu2 = np.array([2.0, 4.0])
sigma2 = np.array([
    [4.0, -3.0],
    [-3.0, 6.0],
]
)

x1 = np.random.multivariate_normal(mu1, sigma1, n1)
x2 = np.random.multivariate_normal(mu2, sigma2, n2)
x = np.vstack([x1, x2])

plt.figure()
plt.scatter(*x1.T,s=2.5)
plt.scatter(*x2.T,s=2.5)

plt.figure()
plt.scatter(*x.T,s=2.5,color='gray')

**Задание 2.1**

Перед вам вектор `x` который содержит в себе некоторое количество двумерных векторов, про которые известно, что они распределены как смесь двух нормальных распределений. Используя формулы EM-алгоритма из лекций, реализуйте алгоритм, и найдите параметры распределений: вектора $\mu_1$ и $\mu_2$, матрицы $\Sigma_1$, $\Sigma_2$, а так же относительное число элементов в первом распределении $\tau$.

Абсолютная точность найденных параметров должна быть лучше $10^{-4}$. Выведите на экран найденные параметры и нарисуйте график разброса точек для вектора `x` на котором цветами покажите оценку вероятности принадлежности данной точки к первому распределению.

*Бонусные баллы* можно получить за успешную попытку решить данную задачу на максимум правдоподобия "в лоб". Т.е. методом прямой максимизации логарифма функции правдоподобия
$$
\arg \max_{\mu_1, \mu_2, \Sigma_1, \Sigma_2, \tau} \sum_{i=1}^{N} \log \left(\tau N(\mathbf{x_i} | \mu_1, \Sigma_1) + (1-\tau) N(\mathbf{x_i} |\mu_2, \Sigma_2)\right)
$$
с использованием методов численной оптимизации из библиотеки `scipy`.

**Задание 2.2**

Решите задание **2.1** используя готовый класс `GaussianMixture` (из библиотеки `sklearn`) для аппроксимации данных смесью нормальных распределений.

## Анализ главных компонент

In [ ]:
from PIL import Image
from glob import glob

images = [Image.open(file).convert(mode='L') for file in sorted(glob('emoji/*.png'))]
print('image count:', len(images))

fig, axes = plt.subplots(2, 3, figsize=(15,10))
axes = axes.reshape(-1)
for i in range(6):
    axes[i].imshow(images[i], cmap='gray')

In [ ]:
shape = images[0].size
data = np.vstack([np.array(image, dtype=float).reshape(-1) for image in images])
mean = data.mean(axis=0)
data -= mean.reshape(1, -1)

In [ ]:
u, s, v = np.linalg.svd(data)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15,10))
axes = axes.reshape(-1)
axes[0].set_title('mean')
axes[0].imshow(mean.reshape(shape), cmap='inferno')
for i in range(5):
    axes[i+1].set_title('component {}'.format(i + 1))
    axes[i+1].imshow(v[i].reshape(shape), cmap='inferno')

In [ ]:
decoder = v[:20, :]
x = data[30, :]
x_hat = np.dot(decoder.T, np.dot(decoder, x))

fig, axes = plt.subplots(1, 2, figsize=(15,10))
axes[0].imshow((x+mean).reshape(shape), cmap='inferno')
axes[1].imshow((x_hat+mean).reshape(shape), cmap='inferno')

In [ ]:
def evaluate_std(decoder_size):
    decoder = v[:decoder_size, :]
    data_hat = np.dot(decoder.T, np.dot(decoder, data.T))
    return np.std(data_hat - data.T)

std = [evaluate_std(s) for s in range(1,data.shape[0])]
plt.figure()
plt.plot(std)
plt.plot(np.sqrt(s))

**Задание 2.3**

Перед вами в массиве `image` хранится набор смайликов. Используя готовый класс `PCA` (из библиотеки `sklearn`) для анализа главных компонент, выполните анализ этого набора данных. Постройте изображения первых пяти главных компонент и среднего.

## Метод опорных векторов

In [ ]:
np.random.seed(43)

x1 = np.random.multivariate_normal(mu1, sigma1, n1)
x2 = np.random.multivariate_normal(mu2, sigma2, n2)
x = np.concatenate([x1, x2])
y = np.concatenate([np.full(x1.shape[0], 0), np.full(x2.shape[0], 1)])

plt.figure()
plt.scatter(*x1.T,s=2.5)
plt.scatter(*x2.T,s=2.5)

**Задание 2.4**

Перед вами данные из задания **2.1**, однако теперь в векторе `y` записан номер распределения к которому принадлежит двухмерная точка. Используя метод опорных векторов `SVC` из пакета `sklearn` с линейным ядром, постройте бинарный классификатор для этих данных. Нанесите раделяющую гиперплоскость (в нашем случае прямую) на график разброса точек.
На графике разброса точек цветом обозначьте вероятность принадлежности к первому классу (распределению) по мнению классификатора. Вам поможет метод `predict_proba()`.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

names = ["length", "width", "size", "conc", "conc1", "asym", "m3long", "m3trans", "alpha", "dist", "class"]
data = pd.read_csv('magic04.csv', names=names)

x = np.asarray(data.iloc[:, :-1])
y = np.asarray(data.iloc[:, [-1]])

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

**Задание 2.5**

Ниже определена функция `evaluate`, она нужна для того, чтобы строить график ROC и печатать на экран различные численные меры эффективности бинарного классификатора для данных из задания **1.4**. К сожалению, некоторые формулы потерялись. Восстановите формулы и примените функцию для оценки бинарных классификаторов на базе логистической регрессии `LogisticRegression` и метода опорных векторов `SVC`.

In [ ]:
import sklearn.metrics

def evaluate(c, x, y):
    y_pred = c.predict(x)
    scores = c.decision_function(x)

    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y, y_pred, labels=['h', 'g']).ravel()
    accuracy  = 
    precision = 
    recall    = 
    specificity = 
    baccuracy = 
    f1 = 
    
    print("Accuracy                  = {:.4f}".format(accuracy))
    print("Ballanced accuracy        = {:.4f}".format(baccuracy))
    print("F1                        = {:.4f}".format(f1))
    print("Precision (PPV)           = {:.4f}".format(precision))
    print("Recall (sensitivity, TPR) = {:.4f}".format(recall))
    print("Specificity (TNR, 1-FPR)  = {:.4f}".format(specificity))
    
    min_score, max_score = np.min(scores), np.max(scores)
    bins = np.linspace(min_score, max_score, 25)
    plt.figure()
    plt.hist(scores[y.reshape(-1) == 'h'], bins, alpha=0.5, label='Hadron (negative)')
    plt.hist(scores[y.reshape(-1) == 'g'], bins, alpha=0.5, label='Gamma (positive)')
    plt.xlabel("Decision function (value)")
    plt.ylabel("Frequency")
    plt.legend()
    
    tpr, fpr, _ = sklearn.metrics.roc_curve(y, scores, pos_label='g')
    auc = sklearn.metrics.roc_auc_score(y, scores)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.title("Receiver operating characteristic")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    print("AUC                       = {:.4f}".format(auc))

In [ ]:
from sklearn.linear_model import LogisticRegression

c = LogisticRegression(random_state=42, solver="newton-cg")
c.fit(x_train, y_train.reshape(-1))

evaluate(c, x_test, y_test)

In [ ]:
from sklearn.svm import SVC

c = SVC(random_state=42)
c.fit(x_train, y_train.reshape(-1))

evaluate(c, x_test, y_test)

In [ ]:
def flat_dict(x):
    if len(x) == 0:
        return dict()
    return {k: np.asarray([e[k] for e in x]) for k in x[0].keys()}

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import make_scorer

C_grid = np.array([1, 10, 100, 1000])

scoring = {
    "auc":       "roc_auc",
    "accuracy":  "accuracy",
    "precision": make_scorer(precision_score, pos_label='g'),
    "recall":    make_scorer(recall_score, pos_label='g'),
}

scores = []

for C in C_grid:
    c = SVC(random_state=42, C=C)
    s = cross_validate(c, x, y.reshape(-1), cv=5, scoring=scoring)
    scores.append(s)
    
scores = flat_dict(scores)

np.set_printoptions(precision=4)
print("fit time = {}".format(scores['fit_time'].mean(axis=1)))
for s in scoring.keys():
    print("{} = {}".format(s, scores["test_{}".format(s)].mean(axis=1)))

**Задание 2.6**

Используя метод кросс-валидации, протестируйте различные ядра для метода опорных векторов и данных из задания **1.4**:
полиномиальное ядро с показателями от 3 до 7 включительно, RBF ядро, сигмоидное ядро.
Получите таблицу значений точности и AUC для всех случаев.

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA

d = PCA()
z_train = d.fit_transform(x_train) 
z_test = d.transform(x_test)

c = SVC(random_state=42)
c.fit(z_train, y_train.reshape(-1))

evaluate(c, z_test, y_test)

In [ ]:
d = PCA()
z = d.fit_transform(x)
plt.plot(np.arange(1, d.singular_values_.shape[0]+1), d.singular_values_, '*')
plt.ylabel("Singular value")
plt.xlabel("n")

**Задание 2.7**

Используя метод кросс-валидации, протестируйте метод опорных векторов для данных из задания **1.4** в следующих ситуациях.
Используйте анализ главных компонент для исходных данных, затем используйте для обечения классификатора только $n$ полученных главных компонент. Исследуйте все возможные значения $n$.
Получите таблицу значений точности и AUC для всех случаев.
Постройте график зависимости AUC от размерности используемых входных предикторов.

**Задание 2.8**

Проделайте такое же исследование, что и в задании **2.7**, но в этот раз исключайте по одной главной компоненте от первой до последней.

**Задание 2.9**

*Бонусные баллы* можно заработать если предъявить классификатор на основе метода опорных векторов для данных из задания **1.4**, который имел бы наилучшее значение AUC (среди всей группы). Попробуйте перебрать различные гиперпараметры: значения параметра $C$, разные типы ядер, применить анализ главных компонент, и т.п.